# 1. 进程

### 进程和线程的区别（资源、调度、开销、通信方式等
* **进程：** 一个具有一定独立功能的程序在一个数据集合上的一次动态执行过程
* **特点：**动态性，并发性，独立性，制约性

**一、基本区别**  
* 进程是（操作系统）**资源分配**的最小单位，线程是（操作系统）**程序执行**的最小单位。

* 进程有自己的独立地址空间，每启动一个进程，系统就会为它分配地址空间，建立数据表来维护代码段、堆栈段和数据段，这种操作非常昂贵。而线程是共享进程中的数据的，使用相同的地址空间，因此CPU切换一个线程的花费远比进程要小很多，同时创建一个线程的开销也比进程要小很多。

* 线程之间的通信更方便，同一进程下的线程共享全局变量、静态变量等数据，而进程之间的通信需要以通信的方式（IPC)进行。不过如何处理好同步与互斥是编写多线程程序的难点。

**二、资源上的区别**
* 计算机的资源分为俩类**计算资源**和**存储资源**
* 进程有自己独立的地址空间，如果一个进程创建了一个子进程，子进程是会在新的地址空间复制父进程的相关数据（也可能一开始不复制，在需要修改的时候再复制（copy on write) 总之，只读资源可以共享，但是写资源必定会复制出新的一快
* 线程是共享进程中的数据的，一个进程中不同的线程是共享同一个地址空间的；不过线程可以有自己独立的堆栈

**三、调度上**（也就是进程，线程之前的上下文切换）  
* 上下文切换是指 一个进程切换到另外一个进程运行的过程，也可以是线程切换
* 所谓的进程、线程的调度问题，就是指通过什么样的方式，对各种需要运行的进程、线程之间进行切换，保证公平效率


### 进程调度算法
* 进程调度算法可以根据目的不同分为三大类：批处理，交互式，实时  
    
    * 批处理：常用在商业领域，希望周转时间越少越好，同时尽可能地减少进程间上下文切换
    * 交互式：希望尽量减少相应时间
    * 实时：希望能满足最后截止时间，要在ddl之前完成任务

**批处理**  
* 先来先服务（first-come first-serverd, FCFS）  
    * CPU资源先到先得，非抢占式算法；
    * 允许一个任务运行任意时长
    * 缺点：没有优先级关系，对于一些优先级/比较紧急的进程无法及时处理
    
    
* 最短作业优先（Shortest Job First，SJF）  
    * 假设每个进程需要运行的时间是已知的
    * 进程所需要运行的时间可以通过历史运行时间确定
    * 运行时间少的先run, 使得总体平均周转时间最小
    * 也就是没有逆序
    
    
* 最短剩余时间优先（Shortest Remaining Time Next，SRTN）  
    * 最短作业优先的抢占版
    * 总是先调用剩余运行时间最短的那个进程先Run
    * 每有一个新的作业，都会和目前已有（包括正在运行的进程）进行比较
    * 如果当前进程的剩余运行时间 > 新的作业，则当前进程挂起，先运行新的作业

**交互式**  
* 轮询调度（round-robin， RR）：轮流占用CPU时间执行
* 优先级队列
* 多级队列 （Compatible TimeSharing System, CTSS）兼容分时系统
* 最短进程优先
(其他调度算法)  
* 保证调度
* 彩票调度
* 公平分享调度（以用户作为出发点）

**实时**  
* 硬实时（ddl前一定得完成）
* 软实时（可以偶尔接受超时）

**RM（Rate Monotonic）速率单调调度**  
* 最佳静态优先级调度
* 通过周期安排优先级
* 周期越短优先级越高
* 先执行周期最短的任务


**EDF（Earliest Deadline First）最早期限调度**  
* 最佳动态优先级调度
* 离deadline越早优先级越高
* 先执行离deadline最近的任务

### 线程调度

**用户级线程和内核级线程** 
（用户级线程和内核级线程的区分的是 线程表所处的位置不同）
* **用户级线程**：在用户态（用户空间）中实现的线程，在用户态的线程，对内核是不可见的，内核并不知道有线程的存在，且在用户态中的线程内部没有 **时钟中断**，所以除非当前线程主动让出CPU资源，否则线程将一值运行（只要得到CPU），无法进行中断；线程用保留在用户进程中，由用户进程负责记录，调度

* **内核级线程**：在内核态中实现的线程，进程中没有独立的线程表，进程表和线程表都保存在内核中，由内核统一处理；在内核态中，线程和进程基本可以不区分，对于线程的调用和进程的调用基本相同；

**用户级线程调度**

* 当线程处于用户线程级时，内核是不知道有线程的存在的，所以只能是进程间的调度来使用，也就是上面的所有进程调度方法都可以是线程调度；  
* 不过需要注意的是，由于用户级线程没有**时钟中断**，除非该线程主动让出CPU，其他线程可以运行任意时间；（在可运行的时间内）
* 允许每个进程有自己定制的调度算法

![](img/pthread.png)

**内核级线程**

* 内核可以直接通过时间片控制线程的运行和挂起；
* 用户级线程和内核级线程之间的主要差别在于**性能**。用户级线程的切换需要少量的机器指令（想象一下Java程序的线程切换）
* 而内核线程需要**完整的上下文切换**，修改内存映像，使高速缓存失效，这会导致了若干数量级的延迟。

**四、通信上**
（IPC）

进程与线程之间的通信，主要是为了解决**同步互斥的问题**  
进程的通信可以分为2大部分：
* 1、控制信息通信，主要是同步互斥
* 2、数据通信，主要是数据的传输、交换、共享等

### 进程通信
####  低级通信


* **忙等互斥：**
    * 屏蔽中断
    * 锁变量
    * 轮询（Peterson 算法，TSL 指令）
    
    

* **睡眠唤醒**：就是在上面出现等待时候，进行睡眠，当资源被释放，进程会被唤醒



* **信号量**：是一种整型计数器；P操作和V操作，P操作-1，当信号量<0时，不可进入临资源，阻塞；V操作，+1，当信号量>=0时，唤醒等待队列中的一个进程



* **互斥量**：信号量的一个简单版本


* **管程（monitor）**：指的是管理共享变量以及对共享变量的操作过程，让它们支持并发



* **内存屏障**：（进程组之间的同步）


#### 高级通信

1. **管道（管道和有名管道）**  
   * 匿名管道：|，匿名管道只能在有关系的进程之间进行通信；  
   * 有名管道：mkfifo 管道名，支持没有关系的进程之间进行通信
   * 匿名管道的创建，是通过系统调用``` int pipe(int fd[2])``` 并返回了两个描述符，一个是管道的读取端描述符 ```fd[0]```，另一个是管道的写入端描述符 ```fd[1]```
   * 管道是存在缓存中的，有数据大小的限制
   * 半双工的，数据只能向一个方向流动；
2. **消息队列（报文队列）**：类型RMQ那种，进程把需要传输的数据放入队列中，需要取数据的一方从队列中取数据即可
    * 有大小限制
    * 通信不及时
    * 有用户态到内核态拷贝的开销
3. **内存共享**：拿出一块虚拟地址空间来，映射到相同的物理内存中。这样这个进程写入的东西，另外一个进程马上就能看到了，都不需要拷贝来拷贝去，传来传去，大大提高了进程间通信的速度。
4. **信号**：对于异常情况下的工作模式，就需要用「信号」的方式来通知进程。
    * 用kill -l可以查看所有信号
    * 信号是一种异步通信机制
    * 进程对信号的处理方式：
        1. **执行默认操作**。Linux 对每种信号都规定了默认操作，例如，上面列表中的 SIGTERM 信号，就是终止进程的意思。Core 的意思是 Core Dump，也即终止进程后，通过 Core Dump 将当前进程的运行状态保存在文件里面，方便程序员事后进行分析问题在哪里。

        2. **捕捉信号**。我们可以为信号定义一个信号处理函数。当信号发生时，我们就执行相应的信号处理函数。

        3. **忽略信号**。当我们不希望处理某些信号的时候，就可以忽略该信号，不做任何处理。有两个信号是应用进程无法捕捉和忽略的，即 SIGKILL 和 SEGSTOP，它们用于在任何时候中断或结束某一进程。
5. **信号量**：主要是用在进程访问共享数据时的互斥；信号量其实是一个整型的计数器；
    * P操作：这个操作会把信号量减去 -1，相减后如果信号量 < 0，则表明资源已被占用，进程需阻塞等待；相减后如果信号量 >= 0，则表明还有资源可使用，进程可正常继续执行。
    * V操作：这个操作会把信号量加上 1，相加后如果信号量 <= 0，则表明当前有阻塞中的进程，于是会将该进程唤醒运行；相加后如果信号量 > 0，则表明当前没有阻塞中的进程；
    * P 操作是用在进入共享资源之前，V 操作是用在离开共享资源之后，这两个操作是必须成对出现的
6. **套接字（socket）**：跨网络与不同主机上的进程之间通信；
    * TCP：绑定的是IP地址和端口
    * UDP：绑定的是IP地址和端口
    * 本地：绑定一个本地文件


https://www.cnblogs.com/xiaolincoding/p/13402297.html

### 线程通信

线程间的通信，主要是用于线程间的同步，没有像进程那样的数据传输通信机制

* 1、 锁机制
互斥锁、条件变量、读写锁和自旋锁

* 2、信号量机制

* 3、信号机制

## 死锁

### 1. 什么是死锁
所谓死锁，是指多个进程在运行过程中因争夺资源而造成的一种僵局，当进程处于这种僵持状态时，若无外力作用，它们都将无法再向前推进。

e.g. 有一个线程A，按照先锁X 再获得锁Y 的的顺序获得锁，而在此同时又有另外一个线程B，按照先锁Y 再锁X的顺序获得锁，这样就会造成死锁问题

### 2. 死锁产生的原因
**a. 竞争资源**

系统中的资源可以分为两类：

* 可剥夺资源，是指某进程在获得这类资源后，该资源可以再被其他进程或系统剥夺，CPU和主存均属于可剥夺性资源；

* 另一类资源是不可剥夺资源，当系统把这类资源分配给某进程后，再不能强行收回，只能在进程用完后自行释放，如磁带机、打印机等。

* 产生死锁中的竞争资源之一指的是竞争不可剥夺资源（例如：系统中只有一台打印机，可供进程P1使用，假定P1已占用了打印机，若P2继续要求打印机打印将阻塞）

* 产生死锁中的竞争资源另外一种资源指的是竞争临时资源（临时资源包括硬件中断、信号、消息、缓冲区内的消息等），通常消息通信顺序进行不当，则会产生死锁

**b. 进程间推进顺序非法**

* 若P1保持了资源R1,P2保持了资源R2，系统处于不安全状态，因为这两个进程再向前推进，便可能发生死锁
* 例如，当P1运行到P1：Request（R2）时，将因R2已被P2占用而阻塞；当P2运行到P2：Request（R1）时，也将因R1已被P1占用而阻塞，于是发生进程死锁

### 3. 死锁的4个必要条件
1. 互斥条件：进程要求对所分配的资源进行排它性控制，即在一段时间内某资源仅为一进程所占用。
2. 请求和保持条件：当进程因请求资源而阻塞时，对已获得的资源保持不放。
3. 不剥夺条件：进程已获得的资源在未使用完之前，不能剥夺，只能在使用完时由自己释放。
4. 环路等待条件：在发生死锁时，必然存在一个进程--资源的环形链。


### 4. 避免死锁的产生
破坏死锁的4个必要条件之一即可打破死锁

1. 资源一次性分配：一次性分配所有资源，这样就不会再有请求了：（破坏请求条件）
2. 只要有一个资源得不到分配，也不给这个进程分配其他的资源：（破坏请保持条件）
3. 可剥夺资源：即当某进程获得了部分资源，但得不到其它资源，则释放已占有的资源（破坏不可剥夺条件）
4. 资源有序分配法：系统给每类资源赋予一个编号，每一个进程按编号递增的顺序请求资源，释放则相反（破坏环路等待条件）以确定顺序获得锁，超时放弃


**银行家算法（死锁避免）**  
* 银行家算法所避免的是进入**不安全状态**，但是不安全状态不等于死锁，不安全状态的范围更大
* 银行家算法使用 四 个数据结构来进行描述：**可利用资源向量， 最大需求矩阵， 分配矩阵， 需求矩阵**， 其中Need[i,j] = Max[i,j] - allocation[i, j]
* 如果所有进程的 Finish[i] =true都满足，则表示系统处于安全状态；否则，系统处于不安全状态。

### 5.死锁检测

**1、每种类型一个资源 的死锁检测**
* 对资源和进程构建一个资源分配图
* 对图使用DFS，如果存在环
* 则存在死锁

![](img/deadlock.png)

**2、每种类型多个资源的死锁检测**  

对资源和进程构建一个资源分配矩阵

* 算法过程
每个进程最开始时都不被标记，执行过程有可能被标记。当算法结束时，任何没有被标记的进程都是死锁进程。


1. 寻找一个没有标记的进程 Pi，它所请求的资源小于等于 A。
2. 如果找到了这样一个进程，那么将 C 矩阵的第 i 行向量加到 A 中，标记该进程，并转回 1。
3. 如果没有这样一个进程，算法终止。

![](img/2.png)

### 死锁恢复

* 利用抢占恢复：抢占已经分配出去的资源给某个进程使用，使用结束后再归还；（不可取）
* 利用回滚恢复：
    * 定期检查流程。进程的检测点意味着进程的状态可以被写入到文件以便后面进行恢复。检测点不仅包含**存储映像**(memory image)，还包含**资源状态**(resource state)。
    * 一种更有效的解决方式是不要覆盖原有的检测点，而是每出现一个检测点都要把它写入到文件中，这样当进程执行时，就会有一系列的检查点文件被累积起来。
    * 为了进行恢复，要从上一个较早的检查点上开始，这样所需要资源的进程会回滚到上一个时间点，在这个时间点上，死锁进程还没有获取所需要的资源，可以在此时对其进行资源分配。

* 通过杀死进程恢复

### 程序运行过程

一个程序，通过编译-->链接-->装载；从硬盘载入内存中才能开始运行  
* 编译：将高级语言编译成机器语言；
* 链接：由链接程序将编译后形成的一组目标模块，以及所需库函数链接在一起，形成一个完整的装入模块；
* 装载：由装入（装载）程序将装入模块装入内存运行；

#### 三种链接的方式

1. 静态链接：在程序运行之前，先将各目标模块及它们所需的库函数链接成一个完整的可执行文件(装入模块)，即得到完整的逻辑地址，之后不再拆开
2. 装入时动态链接：运行前一边装入一边链接
3. 运行时动态链接：有运行需要的才装入；运行时该目标模块时，才对它进行链接，用不到的模块不需要装入内存。

#### 三种装载的方式

1. 绝对装入：编译或汇编时得到绝对地址，即内存物理地址，直接存到对应的物理地址。单道处理系统就是直接操作物理地址，因此绝对装入只适用于单道程序环境

2. 静态重定位装入：装入时将逻辑地址重定位转化为物理地址，多道批处理系统的使用方式。静态重定位的特点是在一个作业装入内存时，必须分配其要求的全部内存空间，如果没有足够的内存，就不能装入该作业。作业一旦进入内存后，在运行期间就不能再移动，也不能再申请内存空间  

3. 动态重定位装入：动态运行时装入，需要一个重定位寄存器支持；程序可以只将需要运行的部分载入内存，动态申请空间，也允许分配离散的内存

## 内存管理

操作系统的内存管理：  
1、一开始没有操作系统时，程序是直接运行在内存上的，每次只能运行一个；  
2、如果需要运行多个程序，可能会出现程序间的覆盖，导致程序运行失败，因为它们都直接运行在物理内存中；  
3、通过虚拟内存，主存+辅存的方式，将需要/正在运行的放在内存中，不需要运行的放在辅存中，可以扩大内存的使用（看起来）  
4、页面的置换，内存碎片的整理  
5、虚拟内存通过CPU中的MMU（内存管理单元）与物理地址建立映射关系；逻辑地址到物理地址的映射  


**连续内存分配和非连续内存分配**  
* 连续内存分配：一个用户进程分配的内存空间是一段连续的空间
* 非连续内存分配：可以为用户进程分配离散的空间

### 连续内存分配

**单一连续内存分配：**内存分为用户区和内核区，仅人一个用户区，每次只能运行一个程序

* 优点: 实现简单;无外部碎片;

* 缺点: 只能用于单用户、单任务的操作系统中;有内部碎片;存储器利用率极低。


**固定分区分配：**将整个用户空间划分为若干个固定大小的分区，在每个分区中只装入一道作业。操作系统需要建立一个数据结构——**分区说明表**，来实现各个分区的分配与回收。每个表项对应一个分区，通常按分区大小排列。每个表项包括对应分区的 大小、起始地址、状态(是否已分配)


当某用户程序要装入内存时，由操作系统内核程序根据用户程序大小检索该表，从中找到一个能满足大小的、未分配的分区，将之分配给该程序，然后修改状态为“已分配”。

优点: 实现简单，无外部碎片。

缺点: 会产生内部碎片，内存利用率低。

![](img/part.png)

**动态分区分配：**不会预先划分内存分区，而是在进程装入内存时， 根据进程的大小动态地建立分区，并使分区的大小正好适合进程的需要

四种算法：  
1. 首次适配：从低地址开始，找到第一个合适的就使用
2. 最佳适配：从低地址开始，遍历全部，找到最合适的内存区域（二者之差最小）作为进程分配的空间
3. 最坏适配：从低地址开始，遍历全部，找到二者之差最大的区域，作为进程分配的空间
4. 邻近适配：从上次分配的位置开始网后，找到一个能用的就使用

### 非连续内存分配
   **操作系统如何管理虚拟地址与物理地址之间的关系——内存分段&&内存分页**

**什么是虚内存**  
虚拟内存是一种**内存管理模式**，它定义了一个连续的虚拟地址空间，为每个进程提供了一个一致的、私有的地址空间，让进程产生一种自己独享主存的感觉；  


使用硬盘(辅存)来扩展内存，是虚拟内存技术的一个结果，虚拟内存空间会存在硬盘中，并且会被内存缓存（按需），有的操作系统还会在内存不够的情况下，将某一进程的内存全部放入硬盘空间中，并在切换到该进程时再从硬盘读取

**虚内存的作用**：  
* 它把主存看作为一个存储在硬盘上的虚拟地址空间的高速缓存，并且只在主存中缓存活动区域（按需缓存）。

* 它为每个进程提供了一个一致的地址空间，从而降低了程序员对内存管理的复杂性。

* 它还保护了每个进程的地址空间不会被其他进程破坏。

**特性**
1. 多次性：一个作业可以分多次被调入内存。多次性是虚拟存储特有的属性  


2.                    对换性：作业运行过程中存在换进换出的过程(换出暂时不用的数据换入需要的数据) 


3.                    虚拟性：虚拟性体现在其从逻辑上扩充了内存的容量(可以运行实际内存需求比物理内存大的应用程序)。虚拟性是虚拟存储器的最重要特征也是其最终目标。虚拟性建立在多次性和对换性的基础上行，多次性和对换性又建立在离散分配的基础上


#### 内存分段

内存分段，分的是**逻辑地址**  
程序是由若干个逻辑分段组成的，如可由代码分段、数据分段、栈段、堆段组成。**不同的段是有不同的属性的，所以就用分段（Segmentation）的形式把这些段分离出来**

#### 分段后的 逻辑地址和物理地址的映射关系


逻辑地址主要是通过**段号和段偏移**找到**段表**来映射到物理地址的**起始地址和段界限**

* **逻辑地址分段：**主要是将一块内存分为**段选择因子**和**段偏移量**；在段选择因子中，包括**段号，修改位，驻留位，保护位,访问位等**
* **段表：**段表是一个二维矩阵，它的索引就是逻辑地址的段号，段表中存储了**段基地址**和**段界限**，就是说，映射到物理内存中，起始地址在哪，以及最多能方位多少空间

分段的不足：  
1. 内存碎片的问题。
2. 内存交换的效率低的问题。

#### 内存分页

**分页是把整个虚拟和物理内存空间切成一段段固定尺寸的大小。**这样一个连续并且尺寸固定的内存空间，我们叫页（Page）【物理地址的分页叫**帧**，逻辑地址的分页叫**页**】。在 Linux 下，每一页的大小为 4KB。

* **逻辑地址分页**：将一个逻辑地址，分成**页号和页偏移量**，其中页号就是页表的索引；如果需要多级页表，还会有一级页号，二级页号等；同样是有修改位，驻留位，保护位，访问位等
* **物理地址分页**：物理地址的一页叫**帧**，物理页的大小和逻辑页的大小是一样的

**这里需要注意的是，对于一个页，物理地址和逻辑地址的大小是一样的，但是物理地址的总大小可以不和逻辑地址的总大小一样，一般情况下逻辑地址会比物理地址更大**

* **页表**：逻辑地址到物理地址的映射就是通过页表完成的；页表同样是一个二维矩阵，含有**逻辑地址的页号和物理地址的帧号**，一般而言每个进程都有自己独立的页表，存放在CPU中的MMU中

**缺页中断处理**

1. 如果内存中存在空闲的物理页，则分配一物理页帧f，将需要访问的页p 装入到物理页f中，修改所对应的页表项内容，把驻留位置为 1 ，物理页帧号置为f，**并重新执行被中断的指令**  


2. 如果内存中不存在空闲的物理页，则，通过某种页面算法，选择一个需要被替换的物理页f，将它所对应的逻辑页q所对应的页表的驻留位置为 0 ，如果期间物理页f被修改过，还需将其写回外存，再将需要访问的物理页p装入f中，修改驻留位和页表；**并重新执行被中断的指令**

![](img/orderbreak.png)

### 多级页表和TLB

**多级页表是用来解决页表空间存储过大的问题，TLB是用来解决页表访问速度的问题，用来加快访问速度**

**多级页表**：在逻辑地址的划分中，页号部分划分为一级页表、二级页表等，最后一级页表才真正存储实际映射的物理地址，这样排列类似一个B+tree  

之所以说多级页表能节省空间，是因为**如果某个一级页表的页表项没有被用到，也就不需要创建这个页表项对应的二级页表了，即可以在需要时才创建二级页表**（程序的局部性原理）

**TLB(Translation Lookaside Buffer,页表缓存、转址旁路缓存、快表)**：专门用来放置最常访问的页表项，速度更快的缓存区

CPU在寻址时，首先会先访问TLB，如果没有命中缓存，再从MMU中查找等

### 页面置换算法

* **最佳置换算法(OPT)：**只具有理论意义的算法，用来评价其他页面置换算法。置换策略是将当前页面中在未来最长时间内不会被访问的页置换出去。


* **先进先出置换算法(FIFO)：**简单粗暴的一种置换算法，没有考虑页面访问频率信息。每次淘汰最早调入的页面。


* **最近最久未使用算法(LRU)：**算法赋予每个页面一个访问字段，用来记录上次页面被访问到现在所经历的时间t，每次置换的时候把t值最大的页面置换出去(实现方面可以采用寄存器或者栈的方式实现)。


* **时钟算法clock(也被称为是最近未使用算法NRU)：**页面设置一个访问位，并将页面链接为一个环形队列，页面被访问的时候访问位设置为1。页面置换的时候，如果当前指针所指页面访问为为0，那么置换，否则将其置为0，循环直到遇到一个访问为位0的页面。


* **改进型Clock算法：**在Clock算法的基础上添加一个修改位，替换时根究访问位和修改位综合判断。优先替换访问位和修改位都是0的页面，其次是访问位为0修改位为1的页面。


* **最少使用算法LFU：**设置寄存器记录页面被访问次数，每次置换的时候置换当前访问次数最少的。



![](img/page.png)

## 分段和分页的区别

1. 分段，分的是逻辑地址，分页，分的是逻辑地址和物理地址，物理地址的帧和逻辑地址的页是对应的
2. 段的大小是可以由用户动态调整的，分页一般是固定大小的；
3. 分段有外碎片，分页没有外碎片
4. 分段是由段号和段偏移映射到物理地址的；分页是由页号和页偏移映射到物理地址的

## 磁盘调度算法

* **先来先服务算法（FCFS）**

* **最短寻道时间优先算法（SSTF）**：选择处理距离当前磁头位置的最短寻道时间的请求，（缺点，较远的地方可能产生饥饿现象）

* **扫描算法（电梯算法，SCAN）**：按照一个方向访问，先处理与磁头移动方向相同的请求，直到顶端，再返过头来往回走，处理其他请求，类似电梯

* **循环扫描算法（C-SCAN）**：先按照一个方向访问，并处理与磁头移动方向相同的请求，直到顶端，然后是直接返回最低端，再开始从低到高扫描，处理请求
* **C look**：C-SCAN的改进版本，只到达最后一个请求的位置为止，不需要到磁盘的最后为止

P.S. 一圈是一个磁道（前后移动，更慢，开销更大），磁道中的每个分区是，扇区


### 上下文切换（进程/线程间切换）

1. 保存处理机上下文，包括程序计数器和其他寄存器。 
2. 更新PCB信息。 
3. 把进程的PCB移入相应的队列，如就绪、在某事件阻塞等队列。 
4. 选择另一个进程执行，并更新其PCB。 
5. 更新内存管理的数据结构。 
6. 恢复处理机上下文。



## 线程切换代价为什么小

1. 进程间的线程共享同一块内存地址，空间是连续的；
2. 线程切换不需要清空CPU和主存中的缓存（如TLB, MMU中的信息等）

## 用户态到内核态的切换

**系统调用，中断，异常**什么导致进程从用户态切换到内核态  

* 所谓的用户态切换到内核态，可以理解为进程之间的切换/线程之间的切换；
* 切换时，需要把用户态中的线程信息，拷贝到内核态中的线程，让内核中的线程代替去执行必要的操作

## 零拷贝技术

**指的是不需要用户空间作为数据中转的技术**，通过系统调用sendfile()实现

**特点**  
* 更少的用户态与内核态的切换
* 不利用 cpu 计算，减少 cpu 缓存伪共享
* 需要注意的是零拷贝适合小文件传输

* **直接 I/O**：对于这种数据传输方式来说，应用程序可以直接访问硬件存储，操作系统内核只是辅助数据传输：这类零拷贝技术针对的是操作系统内核并不需要对数据进行直接处理的情况，数据可以在应用程序地址空间的缓冲区和磁盘之间直接进行传输，完全不需要 Linux 操作系统内核提供的页缓存的支持。


* 在**数据传输**的过程中，避免数据在操作系统内核地址空间的缓冲区和用户应用程序地址空间的缓冲区之间进行拷贝。有的时候，应用程序在数据进行传输的过程中不需要对数据进行访问，那么，将数据从 Linux 的页缓存拷贝到用户进程的缓冲区中就可以完全避免，传输的数据在页缓存中就可以得到处理。在某些特殊的情况下，这种零拷贝技术可以获得较好的性能。Linux 中提供类似的系统调用主要有 mmap()，sendfile() 以及 splice()。


* 对数据在 Linux 的页缓存和用户**进程**的缓冲区之间的传输过程进行优化。该零拷贝技术侧重于灵活地处理数据在用户进程的缓冲区和操作系统的页缓存之间的拷贝操作。这种方法延续了传统的通信方式，但是更加灵活。在Linux中，该方法主要利用了写时复制技术。
